In [ ]:
from summoner_data import SummonerData
from map_helper import MapHelper

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from collab_filter import ChampionsDataset, DotProduct, train_and_evaluate_model
from torch import optim

In [ ]:
sd = SummonerData()
mh = MapHelper()

In [7]:
df, le_user, le_champion = load_clean_df()

# train test split
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader= DataLoader(test_dataset, batch_size=20, shuffle=True)

num_summoners = len(df['puuid'].unique())
num_champions = len(df['champ_id'].unique())
num_factors = 5

In [8]:
model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader)

Epoch 1, Loss: 0.41394319937953467
Test MSE (Rounded Predictions): 0.5674281120300293
Epoch 2, Loss: 0.4137612237583233
Test MSE (Rounded Predictions): 0.5674281120300293
Epoch 3, Loss: 0.41403676622276064
Test MSE (Rounded Predictions): 0.5674280524253845
Epoch 4, Loss: 0.41407720231557194
Test MSE (Rounded Predictions): 0.5674281120300293
Epoch 5, Loss: 0.41375818237473694
Test MSE (Rounded Predictions): 0.5674280524253845
Epoch 6, Loss: 0.4137738337999658
Test MSE (Rounded Predictions): 0.5674280524253845
Epoch 7, Loss: 0.4138284292402147
Test MSE (Rounded Predictions): 0.5674281120300293
Epoch 8, Loss: 0.41354504669014414
Test MSE (Rounded Predictions): 0.5674280524253845
Epoch 9, Loss: 0.413831935351408
Test MSE (Rounded Predictions): 0.5674280524253845
Epoch 10, Loss: 0.413425087438354
Test MSE (Rounded Predictions): 0.5669209957122803


In [9]:
all_champions = torch.tensor([i for i in range(num_champions)])
user_id = torch.tensor([1] * len(all_champions)) 
predicted_ratings = model(user_id, all_champions)
predicted_ratings

tensor([3.5016e-01, 1.7322e-01, 2.5037e-01, 5.0010e-01, 3.1984e-01, 3.9973e-01,
        9.4561e-01, 2.7043e-01, 3.7404e-04, 6.9337e-01, 1.8963e-01, 3.7962e-01,
        8.5791e-01, 7.7112e-01, 2.3370e-01, 6.2008e-01, 2.4968e-02, 9.1674e-01,
        9.7676e-01, 8.5318e-01, 4.5410e-01, 4.6505e-01, 1.8016e-03, 8.8444e-01,
        7.4348e-01, 3.1627e-02, 2.8242e-01, 9.3086e-01, 1.4915e-01, 7.5645e-01,
        3.3067e-01, 9.5568e-01, 9.4909e-02, 4.2867e-01, 5.8897e-01, 8.5327e-01,
        7.5764e-01, 7.6923e-01, 2.5407e-03, 2.6864e-01, 6.5560e-01, 9.7813e-01,
        6.5598e-01, 6.6045e-01, 8.8976e-01, 2.1230e-01, 9.3700e-01, 9.1078e-02,
        2.2502e-01, 7.9873e-01, 4.0019e-01, 2.2707e-02, 8.5919e-01, 1.5486e-03,
        7.6175e-01, 2.6127e-01, 9.4151e-01, 4.2864e-01, 6.7976e-01, 9.1953e-01,
        8.0920e-01, 1.4027e-02, 8.7490e-01, 8.2905e-01, 7.4673e-01, 9.8803e-01,
        1.2217e-01, 5.4593e-01, 2.0461e-01, 4.2808e-01, 1.5167e-01, 8.5978e-01,
        9.5442e-01, 9.8730e-01, 8.1268e-

In [20]:
top_ratings, top_indices = torch.topk(predicted_ratings, k=3)

# Convert tensor indices to integers if needed
top_indices_list = top_indices.tolist()

In [23]:
for idx in top_indices_list:
    print(f"WE PRESCRIBE THAT YOU PLAY: {champ_order[idx]}")

WE PRESCRIBE THAT YOU PLAY: Lucian
WE PRESCRIBE THAT YOU PLAY: Talon
WE PRESCRIBE THAT YOU PLAY: Nasus


In [ ]:
# WHAT IS YOUR SUMMONER ID? 
# YOU ARE NOT ON FILE. GENERATING...
# YOU ARE ON FILE. MODELLING...
# YOUR CURRENT MOST PLAYED IS ... WE RECOMMEND...

In [4]:
le_user.inverse_transform([1])

array(['0EMIdPXGT4kuqhWoMhAyVcfuaHlRk87Ofm-KuAtYVKkcxxls6RNZ-Go1XJcVX6n9jsI30dmzNQ_PVw'],
      dtype=object)

In [26]:
print("Print your in game name as such: Fred#NA1...")

Print your in game name as such: Fred#NA1...


In [6]:
import os
RIOT_API_KEY = os.environ.get("RIOT_API_KEY")
RIOT_API_KEY

'RGAPI-67a9e6ce-05b9-41a9-a84f-6f4ac2b7d2ff'

In [1]:
# Puts it all together
import os

import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from collab_filter import ChampionsDataset, DotProduct, train_and_evaluate_model
from map_helper import MapHelper
from summoner_data import SummonerData

PROJECT_ROOT = os.getenv("PROJECT_ROOT")

map_helper = MapHelper()
summoner_data = SummonerData()
rewrite = False

# Prompt in game name
puuid = map_helper.get_puuid_mapping()

In [ ]:
#Check if data exists as json
puuid_path = os.path.join(PROJECT_ROOT, f"src/summoner_data/{puuid}.json")
if not os.path.exists(puuid_path):
    rewrite = True
    summoner_data.dump_matches_for_puuid(puuid)

# Get the rating data
_ = summoner_data.aggregate_json(True)
df, le_user, le_champion = summoner_data.load_clean_df_encoders(True)

# Train a model
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=True)

num_summoners = len(df["puuid"].unique())
num_champions = len(df["champ_id"].unique())
num_factors = 5

model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader)
# Get relevant row
user_idx = le_user.transform([puuid])

Re-aggregating json...
Loading new ratings...
User Embeddings Before Training:
tensor([[ 2.3690,  0.2873,  0.8923, -0.2729, -0.6466],
        [-0.1189, -0.8681, -1.4192,  1.1580,  0.5485],
        [ 0.0672, -1.2097,  0.0176,  0.6157, -0.6444],
        [-0.2056,  1.5893, -0.2634, -0.2865, -1.3269],
        [-0.1026,  0.1347,  0.6852, -1.3550, -0.7912]],
       grad_fn=<SliceBackward0>)
Item Embeddings Before Training:
tensor([[ 0.0249,  0.3249,  0.7492,  0.1573, -1.8750],
        [ 0.4295, -0.5778,  0.6961,  1.0842,  1.3802],
        [ 0.5694,  0.8366,  1.4554, -0.2050,  1.1136],
        [ 0.3418, -0.8622, -0.1369, -0.1809,  1.6811],
        [-1.3125, -0.0887, -1.0633,  0.1687, -0.6163]],
       grad_fn=<SliceBackward0>)
Epoch 1, Loss: 4.771667809925269
Test MSE (Rounded Predictions): 4.563136100769043
Epoch 2, Loss: 4.5973392962223265
Test MSE (Rounded Predictions): 4.414419651031494
Epoch 3, Loss: 4.441386195261087
Test MSE (Rounded Predictions): 4.31016731262207
Epoch 4, Loss: 4.2897

In [15]:
all_champions

tensor([0.6734, 0.8631, 0.9283, 0.9599, 0.6041, 0.9548, 0.5921, 0.1938, 0.2579,
        0.9226, 0.6161, 0.1246, 0.8867, 0.4801, 0.0662, 0.0833, 0.6780, 0.1933,
        0.1752, 0.6738, 0.3451, 0.8650, 0.2480, 0.7855, 0.0153, 0.6244, 0.1878,
        0.5202, 0.2650, 0.2060, 0.3161, 0.4936, 0.1617, 0.4068, 0.3938, 0.9145,
        0.8185, 0.8273, 0.4854, 0.8114, 0.6261, 0.0647, 0.1143, 0.6823, 0.4623,
        0.8374, 0.6811, 0.1737, 0.1535, 0.2488, 0.0855, 0.6467, 0.4845, 0.8451,
        0.0297, 0.7419, 0.7130, 0.9670, 0.6072, 0.5096, 0.1103, 0.1724, 0.7426,
        0.4234, 0.4896, 0.9261, 0.3896, 0.6541, 0.3440, 0.3871, 0.5896, 0.7940,
        0.8262, 0.6594, 0.9704, 0.2902, 0.6415, 0.9639, 0.1097, 0.1480, 0.2111,
        0.8691, 0.2314, 0.8650, 0.3334, 0.4526, 0.7835, 0.4966, 0.9833, 0.6980,
        0.8537, 0.6218, 0.8743, 0.7042, 0.4063, 0.5714, 0.8051, 0.7978, 0.3211,
        0.4429, 0.4833, 0.2745, 0.2715, 0.1345, 0.2682, 0.3595, 0.1886, 0.1775,
        0.3104, 0.5436, 0.5687, 0.0617, 

In [25]:
import numpy as np
# Get pred
all_champions = torch.tensor([i for i in range(num_champions)])
# all_champions = torch.tensor(df['champ_id'].unique(), dtype=torch.long)
# user_id = torch.tensor(list(user_idx) * len(all_champions))
user_id = torch.tensor([1] * len(all_champions))
predicted_ratings = model(user_id, all_champions)
top_ratings, top_indices = torch.topk(predicted_ratings, k=3)
top_indices_list = top_indices.tolist()

# Load champ order
champ_map = map_helper.get_champ_id_to_name()
champ_order = [
    champ_map.get(str(x)) for x in le_champion.inverse_transform(range(num_champions))
]
for idx in top_indices_list:
    print(f"WE PRESCRIBE THAT YOU PLAY: {champ_order[idx]}")

WE PRESCRIBE THAT YOU PLAY: Viktor
WE PRESCRIBE THAT YOU PLAY: Yone
WE PRESCRIBE THAT YOU PLAY: Malphite


In [7]:
model.summoner_factors.weight[:5]

tensor([[-0.8350,  0.9490,  0.3413,  1.0128,  1.0120],
        [ 0.8737, -0.4630, -1.4928, -0.2391,  0.9873],
        [ 1.1687,  0.3299,  0.0453, -0.5464, -1.0530],
        [-0.5251, -0.1215,  1.0812, -0.3236,  0.3960],
        [-1.0846,  0.1898,  0.2224,  0.2006, -0.1719]],
       grad_fn=<SliceBackward0>)

In [6]:
user_id = torch.tensor([3] * len(all_champions))
predicted_ratings = model(user_id, all_champions)
predicted_ratings

tensor([0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 

In [46]:
import requests

RIOT_API_KEY = os.environ.get("RIOT_API_KEY")
game_name = "BSIZZLEMONEY"
tag_line = "0000"

url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
headers = {"X-Riot-Token": RIOT_API_KEY}

response = requests.get(url, headers=headers)
print(response.status_code)
print(response.json())


400
{'status': {'message': 'Unknown apikey', 'status_code': 400}}


In [12]:
le_champion.inverse_transform(range(num_champions))

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 67, 68, 69, 72, 74, 75, 76, 77, 78, 79,
       80, 81, 82, 83, 84, 85, 86, 89, 90, 91, 92, 96, 98, 99, 101, 102,
       103, 104, 105, 106, 107, 110, 111, 112, 113, 114, 115, 117, 119,
       120, 121, 122, 126, 127, 131, 133, 134, 136, 141, 142, 143, 145,
       147, 150, 154, 157, 161, 163, 164, 166, 200, 201, 202, 203, 221,
       222, 223, 233, 234, 235, 236, 238, 240, 245, 246, 254, 266, 267,
       268, 350, 360, 412, 420, 421, 427, 429, 432, 497, 498, 516, 517,
       518, 523, 526, 555, 711, 777, 799, 800, 875, 876, 887, 888, 893,
       895, 897, 901, 902, 910, 950], dtype=object)

In [ ]:
from map_helper import get_champ_id_to_name
map = get_champ_id_to_name()
champ_order = [map.get(str(x)) for x in le_champion.inverse_transform(range(num_champions))]

In [ ]:
"""
for epoch in range(epochs):
    # set in training mode
    model.train()
    # some metric of keeping track of losses
    loss = 0
    criterion = nn.MSELoss()
    # these are tensors, in this case the locations of whichever data batch is being handed over
    for user_id, champ_id, ratings in train_loader:
        # Just empties the gradients before
        optimizer.zero_grad()
        # this gets passed into the forward function
        preds = model(user_id, champ_id)
        # compares yhat to y
        loss = criterion(preds, ratings)

        # Computes the dloss/dx for every parameter x... what is x?... so something like x.grad += dloss/dx 
        loss.backward()
        # updates the x value using x.grad... so x += -lr * x.grad
        optimizer.step()
        loss += loss.item()

        
        
        
        
        
Synapse that transforms the X into something... and each node then puts an activation function (think sigmoid)
Think of this as the W in the matmul, except this time we're literally just amking it a... dot product?
gradient descent is saying do we want W to go... up or down? gives us a direction to minimize cost
Back prop is essentially saying that we are starting from the end and the "error" term of gradient descent is passed on to the node which contributes the most
every example gets a vote on which way is downhill        
"""

One layer
    Synapse takes the dot product 

Main confusing parts are loss.backward() and optimizer.step() (what are the params excatly?) as well as what's going on in the forward.... like what are user_id and champ_id representing? 

In [ ]:
import json
import os
import pickle as pkl
import time
from collections import defaultdict
from typing import Any, Dict, List, Optional

import pandas as pd
import requests
from sklearn.preprocessing import LabelEncoder

from map_helper import get_champ_id_to_name

API_KEY = os.getenv("RIOT_API_KEY")
PROJECT_ROOT = "/Users/brtnl/OneDrive/Desktop/code/lol_main_recommender"
HEADERS = {"X-Riot-Token": API_KEY}
REGION = "na1"
MATCH_REGION = "americas"

pkl_path = os.path.join(PROJECT_ROOT, "src/data.pkl")

json_folder_path = os.path.join(PROJECT_ROOT, "src/summoner_data/")

player_champions = defaultdict(
    lambda: defaultdict(int)
)  # {player_id: {champion_id: count}}
for json_file in os.listdir(json_folder_path):
    puuid = json_file.split(".json")[0]
    with open(os.path.join(json_folder_path, json_file), "r") as f:
        match_data = json.load(f)
    for match in match_data:
        participants = match.get("info", {}).get("participants", [])
        for participant in participants:
            if participant.get("puuid") == puuid:
                # Store information to get rating
                champion_id = participant.get("championId")
                win = participant.get("win")
                kills = participant.get("kills")
                deaths = participant.get("deaths")
                assists = participant.get("assists")

                # rating = (win_percentage_weight * win_percentage) + (kda_weight * kda) + (matches_played_weight * matches_played)
                player_champions[puuid][champion_id] += 1
                player_champions[puuid][f"{champion_id}_kills"] += kills
                player_champions[puuid][f"{champion_id}_assists"] += assists
                player_champions[puuid][f"{champion_id}_deaths"] += deaths
                if win == 1:
                    player_champions[puuid][f"{champion_id}_wins"] += 1

In [26]:
player_champion_stats = defaultdict(lambda: defaultdict(int))
# calculate kda
for puuid, champ_data in player_champions.items():
    for champion_id, count in champ_data.items():
        # If the key is equal to the id, can construct the stats
        if isinstance(champion_id, int):
            kills = player_champions[puuid][f"{champion_id}_kills"]
            assist = player_champions[puuid][f"{champion_id}_assists"]
            deaths = player_champions[puuid][f"{champion_id}_deaths"]
            wins = player_champions[puuid].get(f"{champion_id}_wins", 0)
            win_pct = (wins / count) * 100 if count > 0 else 0
            kda = (kills + assists) / (deaths if deaths != 0 else 1)

            player_champion_stats[puuid][champion_id] = kda + win_pct + count

In [27]:
player_champion_stats

defaultdict(<function __main__.<lambda>()>,
            {'-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ': defaultdict(int,
                         {55: 3.0,
                          84: 3.0769230769230766,
                          235: 103.22222222222223,
                          8: 102.55555555555556,
                          67: 75.1661750245821,
                          92: 109.57157640565711,
                          18: 6.0,
                          126: 31.272727272727273,
                          5: 54.45454545454545,
                          221: 12.0,
                          98: 102.2,
                          91: 101.71428571428571,
                          21: 104.55555555555556,
                          523: 111.5,
                          64: 8.5,
                          238: 102.5,
                          37: 101.5,
                          266: 109.5,
                          51: 7.0,
                          76: 3.